<h1> MTPB Data: step-by-step instructions --> incremental code generation </h1>

<p>
generate code using MTPB's "prompt" instructions, alternating between prompts and code as shown below:

Input: prompt1
Output: code1

Input: code1 + prompt2
Output: code2

Input: code1 + code2 + prompt3
Output: code3</p>

<h1>Exploration of the reports and outputs<h1>

In [15]:
import pandas as pd

report_baseline = pd.read_csv('data/MTBP/reports/reports_MTBP_Processed_Greedy_solutions.csv')
report_lbl_w = pd.read_csv('data/MTBP/reports/reports_line_by_line_with_context_Sampling.csv')
report_lbl_wo = pd.read_csv('data/MTBP/reports/reports_line_by_line_without_context_Sampling.csv')
report_str_S = pd.read_csv('data/MTBP/reports/reports_structural_mtbp_with_context_Sampling.csv') 
report_str_G = pd.read_csv('data/MTBP/reports/reports_structural_mtbp_with_context_Greedy.csv') 

In [51]:
import ast

def count_pass_one_completed(df):
    count_succes = 0
    indexes_successful = []
    number_of_success = []
    for i in range(len(df)):
        test = ast.literal_eval(df.iloc[i]['Pass_one'])
        count_succes+=sum(test)
        if sum(test) >= 1:
            indexes_successful.append(i)
            number_of_success.append(sum(test))
    
    return count_succes, number_of_success, indexes_successful

In [55]:
_,success_base, list_baseline = count_pass_one_completed(report_baseline)
_,success_lbl_w,list_lbl_w = count_pass_one_completed(report_lbl_w)
_,success_lbl_wo,list_lbl_wo = count_pass_one_completed(report_lbl_wo)
_,success_str_S,list_stru_S = count_pass_one_completed(report_str_S)
_,success_str_G,list_stru_G = count_pass_one_completed(report_str_G)

print('report')
print(f'number of Pass One for the baseLine: {count_pass_one_completed(report_baseline)[0]} out of 575')
print(f'number of Pass One for the line by line with context: {count_pass_one_completed(report_lbl_w)[0]} out of 575')
print(f'number of Pass One for the line by line without context: {count_pass_one_completed(report_lbl_wo)[0]} out of 575')
print(f'number of Pass One for the structural Sampling generation: {count_pass_one_completed(report_str_S)[0]} out of 575')
print(f'number of Pass One for the structural Greddy generation: {count_pass_one_completed(report_str_G)[0]} out of 575')

report
number of Pass One for the baseLine: 88.0 out of 575
number of Pass One for the line by line with context: 10.0 out of 575
number of Pass One for the line by line without context: 33.0 out of 575
number of Pass One for the structural Sampling generation: 52.0 out of 575
number of Pass One for the structural Greddy generation: 62.0 out of 575


In [56]:
print(list_baseline)
print(success_base)
print(list_stru_G)
print(success_str_G)

diff_b_to_sbs = []
for i in list_baseline:
    if i not in list_stru_G:
        diff.append(i)

[3, 4, 21, 25, 26, 28, 29, 33, 35, 36, 51, 53, 54, 55, 56, 58, 59, 67, 72, 79, 85, 89, 90, 91, 93, 97, 98, 100, 103, 107, 108]
[1.0, 1.0, 2.0, 5.0, 5.0, 2.0, 5.0, 5.0, 3.0, 5.0, 5.0, 1.0, 4.0, 4.0, 4.0, 5.0, 1.0, 1.0, 5.0, 5.0, 1.0, 2.0, 3.0, 1.0, 1.0, 3.0, 1.0, 2.0, 1.0, 1.0, 3.0]
[25, 26, 28, 31, 40, 48, 51, 53, 54, 55, 56, 67, 72, 89, 92, 93, 94, 106, 108, 113]
[5.0, 2.0, 5.0, 5.0, 2.0, 5.0, 5.0, 5.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 5.0, 1.0, 2.0, 2.0, 3.0, 1.0]


<h2>Inspection of the output :</h2>
Inspection of the following elements : [3, 4, 21, 29, 33, 35, 36, 58, 59, 79, 85, 90, 91, 97, 98, 100, 103, 107] or What worked with the baseline that doesn't work with the structural greedy step by step generation

In [26]:
import pandas as pd
base = pd.read_csv('data/MTBP/baseline/MTBP_Processed_Greedy_solutions.csv')
step_by_step = pd.read_csv('data/MTBP/step_by_step/structural_mtbp_with_context_Greedy.csv')
diff

[3, 4, 21, 29, 33, 35, 36, 58, 59, 79, 85, 90, 91, 97, 98, 100, 103, 107]

In [58]:
diff_sbs_to_b = []
for i in list_stru_G:
    if i not in list_baseline:
        diff_sbs_to_b.append(i)

diff_sbs_to_b

[31, 40, 48, 92, 94, 106, 113]

<h1>Inspection of the output : </h1>
Inspection of the following elements : [31, 40, 48, 92, 94, 106, 113] or What worked with the step by step that didn't work with the baseline

In [66]:
index = 113
print('step by step :')
print(step_by_step.iloc[index]['code_test'])
print('===============')
print('base :')
print(base.iloc[index]['code_test'])

step by step :
def matrix_size(A):
	#Assign the matrix "{A}" to a variable named "my_matrix".
	my_matrix = A
	#Calculate the number of rows of my_matrix and store as row_number.
	row_number = len(my_matrix)
	#Calculate the number of columns of my_matrix and store as column_number.
	column_number = len(my_matrix[0])
	#Calculate the sum of row_number and column_number and print the result.
	return(row_number * column_number)
base :
def matrix_size(A):
	"""Calculate the sum of row and column numbers."""
	return A.shape[0] * A.shape[1]


In [65]:
baseline = list_baseline
step_by_step = list_stru_G

print(baseline)
print(step_by_step)

[4, 29, 33, 56, 72, 91, 93, 107]
[25, 26, 28, 31, 40, 48, 51, 53, 54, 55, 56, 67, 72, 89, 92, 93, 94, 106, 108, 113]


In [75]:
list_c = ast.literal_eval(report_str_S.iloc[4]['candidate'])

In [76]:
print(list_c[0][0])

def count_negative_numbers(A):
	#Assign the list of numbers "{A}" to a variable named "my_numbers".
	my_numbers = list(A)
	#Count the number of negative numbers in the list as "n_neg".
	n_neg = 0
	#Count the number of positive numbers in the list as "n_pos".
	n_pos = 0
	#Print out the larger number of those two.
	if max(my_numbers) > min(my_numbers):
		for i in my_numbers:
			if i < 0:
				n_neg += 1


In [4]:


def calculate_comment_ratio(func_code):
    # Split the code into lines
    lines = func_code.strip().split('\n')
    # Initialize counters
    code_lines = 0
    comment_lines = 0
    # Iterate through lines
    for line in lines:
        stripped_line = line.strip()
        # Ignore empty lines
        if stripped_line == "":
            continue
        
        # If the line starts with '#', it is a comment line
        if stripped_line[0] == '#':
            comment_lines += 1
        else:
            # Otherwise, it is a line of code
            code_lines += 1

    # Check if there are no lines of code to prevent division by zero
    if code_lines == 0:
        return "No lines of code present"
    
    # Calculate and return the ratio
    return comment_lines / code_lines

def comment_ratio(df):
    #Initialize counter
    ratio = 0
    for i in range(len(df)):
        ratio+=calculate_comment_ratio(df.iloc[i]['gen_code'])
    return ratio/len(df)

def best_candidate(df):
    # Compute the average rate of Pass_one for each candidate
    df['average'] = df['Pass_one'].apply(lambda x: sum(x) / len(x) if len(x) != 0 else 0)

    # Identify the candidate with the highest average rate
    best_candidate = df.loc[df['average'].idxmax()]['candidate']

    return best_candidate